In [1]:
import torch
import torch.nn.functional as F

import numpy as np
import os

from ImgTransformer import ImgTransformer
from Models import getModels

In [2]:
models_dict = getModels()

C:\Users\jeman\Anaconda3\envs\trail_workshop_clean_deepdream\lib\site-packages\pytorch_lightning\core\saving.py:209: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['non_linear_evaluator.block_forward.2.weight', 'non_linear_evaluator.block_forward.2.bias']
  rank_zero_warn(


In [3]:
img_transformer = ImgTransformer()

res = img_transformer.get_transformed_img('../pneumonia/chest_xray/test/NORMAL/IM-0001-0001.jpeg')
img, converted_img, transformed_img, input_ = res

In [4]:
output = models_dict['resnet50_simclr_crop_12'](input_)
# output = F.softmax(output[1], dim=1) # swav
output = F.softmax(output, dim=1) # simclr
prediction_score, pred_label_idx = torch.topk(output, 1)

print('Predicted: NORMAL (', prediction_score.squeeze().item(), ')')

Predicted: NORMAL ( 0.5499179363250732 )


C:\Users\jeman\Anaconda3\envs\trail_workshop_clean_deepdream\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Confusion Matrices

In [6]:
torch.manual_seed(0)
np.random.seed(0)
confusion_matrices = {}

for name, model in models_dict.items():
    print('begin model', name)
    confusion_matrix = {}
    
    test_dir_path = '../pneumonia/chest_xray/test/'
    confusion_matrix['true_positive'] = 0
    confusion_matrix['false_negative'] = 0
    pneumonia_dir_path = test_dir_path + 'PNEUMONIA/'
    for img_path in os.listdir(pneumonia_dir_path):
        if 'crop' in name:
            current_input = img_transformer.get_transformed_img(pneumonia_dir_path + img_path, normalize=True, crop=True)[-1]
        else:
            current_input = img_transformer.get_transformed_img(pneumonia_dir_path + img_path, normalize=False, crop=False)[-1]
        with torch.no_grad():
            current_output = model(current_input)
        if 'swav' in name: current_output = current_output[1]
        res = torch.nn.functional.softmax(current_output[0],dim=0)[:2]
        if res[0] > res[1]:
            confusion_matrix['false_negative'] += 1
        else:
            confusion_matrix['true_positive'] += 1
    print('total', len(os.listdir(pneumonia_dir_path)), 'true_positive', confusion_matrix['true_positive'], 'false_negative', confusion_matrix['false_negative'])

    confusion_matrix['true_negative'] = 0
    confusion_matrix['false_positive'] = 0
    normal_dir_path = test_dir_path + 'NORMAL/'
    for img_path in os.listdir(normal_dir_path):
        if 'crop' in name:
            current_input = img_transformer.get_transformed_img(normal_dir_path + img_path, normalize=True, crop=True)[-1]
        else:
            current_input = img_transformer.get_transformed_img(normal_dir_path + img_path, normalize=False, crop=False)[-1]
        with torch.no_grad():
            current_output = model(current_input)
        if 'swav' in name: current_output = current_output[1]
        res = torch.nn.functional.softmax(current_output[0],dim=0)[:2]
        if res[0] > res[1]:
            confusion_matrix['true_negative'] += 1
        else:
            confusion_matrix['false_positive'] += 1
    print('total', len(os.listdir(normal_dir_path)), 'false_positive', confusion_matrix['false_positive'], 'true_negative', confusion_matrix['true_negative'])
    
    print('accuracy', (confusion_matrix['true_negative'] + confusion_matrix['true_positive']) / (len(os.listdir(normal_dir_path))+len(os.listdir(pneumonia_dir_path))))
    print('precision', confusion_matrix['true_positive'] / (confusion_matrix['true_positive'] + confusion_matrix['false_positive']))
    print('recall', confusion_matrix['true_positive'] / (confusion_matrix['true_positive'] + confusion_matrix['false_negative']))
        
    confusion_matrices[name] = confusion_matrix
    print()

begin model resnet50_simclr_5
total 390 true_positive 389 false_negative 1
total 234 false_positive 102 true_negative 132
accuracy 0.8349358974358975
precision 0.7922606924643585
recall 0.9974358974358974

begin model resnet50_swav_13
total 390 true_positive 390 false_negative 0
total 234 false_positive 116 true_negative 118
accuracy 0.8141025641025641
precision 0.7707509881422925
recall 1.0

begin model resnet50_simclr_crop_12
total 390 true_positive 390 false_negative 0
total 234 false_positive 94 true_negative 140
accuracy 0.8493589743589743
precision 0.8057851239669421
recall 1.0

begin model resnet50_swav_crop_10
total 390 true_positive 389 false_negative 1
total 234 false_positive 96 true_negative 138
accuracy 0.844551282051282
precision 0.8020618556701031
recall 0.9974358974358974



## Confusion Matrices (reverse crop)

Cropped images to models trained with not cropped images and inversely

In [8]:
torch.manual_seed(0)
np.random.seed(0)
confusion_matrices = {}

for name, model in models_dict.items():
    print('begin model', name)
    confusion_matrix = {}
    
    test_dir_path = '../pneumonia/chest_xray/test/'
    confusion_matrix['true_positive'] = 0
    confusion_matrix['false_negative'] = 0
    pneumonia_dir_path = test_dir_path + 'PNEUMONIA/'
    for img_path in os.listdir(pneumonia_dir_path):
        if 'crop' in name:
            current_input = img_transformer.get_transformed_img(pneumonia_dir_path + img_path, normalize=True, crop=False)[-1]
        else:
            current_input = img_transformer.get_transformed_img(pneumonia_dir_path + img_path, normalize=False, crop=True)[-1]
        with torch.no_grad():
            current_output = model(current_input)
        if 'swav' in name: current_output = current_output[1]
        res = torch.nn.functional.softmax(current_output[0],dim=0)[:2]
        if res[0] > res[1]:
            confusion_matrix['false_negative'] += 1
        else:
            confusion_matrix['true_positive'] += 1
    print('total', len(os.listdir(pneumonia_dir_path)), 'true_positive', confusion_matrix['true_positive'], 'false_negative', confusion_matrix['false_negative'])

    confusion_matrix['true_negative'] = 0
    confusion_matrix['false_positive'] = 0
    normal_dir_path = test_dir_path + 'NORMAL/'
    for img_path in os.listdir(normal_dir_path):
        if 'crop' in name:
            current_input = img_transformer.get_transformed_img(normal_dir_path + img_path, normalize=True, crop=False)[-1]
        else:
            current_input = img_transformer.get_transformed_img(normal_dir_path + img_path, normalize=False, crop=True)[-1]
        with torch.no_grad():
            current_output = model(current_input)
        if 'swav' in name: current_output = current_output[1]
        res = torch.nn.functional.softmax(current_output[0],dim=0)[:2]
        if res[0] > res[1]:
            confusion_matrix['true_negative'] += 1
        else:
            confusion_matrix['false_positive'] += 1
    print('total', len(os.listdir(normal_dir_path)), 'false_positive', confusion_matrix['false_positive'], 'true_negative', confusion_matrix['true_negative'])
    
    print('accuracy', (confusion_matrix['true_negative'] + confusion_matrix['true_positive']) / (len(os.listdir(normal_dir_path))+len(os.listdir(pneumonia_dir_path))))
    print('precision', confusion_matrix['true_positive'] / (confusion_matrix['true_positive'] + confusion_matrix['false_positive']))
    print('recall', confusion_matrix['true_positive'] / (confusion_matrix['true_positive'] + confusion_matrix['false_negative']))
    
    confusion_matrices[name] = confusion_matrix
    print()

begin model resnet50_simclr_5
total 390 true_positive 390 false_negative 0
total 234 false_positive 234 true_negative 0
accuracy 0.625
precision 0.625
recall 1.0

begin model resnet50_swav_13
total 390 true_positive 390 false_negative 0
total 234 false_positive 233 true_negative 1
accuracy 0.6266025641025641
precision 0.6260032102728732
recall 1.0

begin model resnet50_simclr_crop_12
total 390 true_positive 388 false_negative 2
total 234 false_positive 105 true_negative 129
accuracy 0.8285256410256411
precision 0.7870182555780934
recall 0.9948717948717949

begin model resnet50_swav_crop_10
total 390 true_positive 381 false_negative 9
total 234 false_positive 126 true_negative 108
accuracy 0.7836538461538461
precision 0.7514792899408284
recall 0.9769230769230769

